In [2]:
import pandas as pd
import numpy as np

In [3]:
# crear links
allbusstop_df = pd.read_csv('./../data/allbusstop.csv', sep=',')

links_df = pd.DataFrame()
ini = allbusstop_df['id'].iloc[:-1].values
end = allbusstop_df['id'].iloc[1:].values
links_df['link'] = [f'{i}:{e}' for i, e in zip(ini, end )]
links_df

,link
0,0:1
1,1:2
2,2:3
3,3:4
4,4:5
...,...
149,149:150
150,150:151
151,151:152
152,152:153


a
<img src='graficos/trayectoria_bus_stops.png' width='600' height='300'>

In [4]:
# leer puntos 
monitoringdf = pd.read_csv('../data/_Process_monitoring__202303251137.csv', sep=',')
monitoringdf = monitoringdf[['id', 'latitude','longitude','date','time', 'lap', 'navigation', 'vehicle_id']]
monitoringdf

,id,latitude,longitude,date,time,lap,navigation,vehicle_id
0,13173398,-13.549597,-71.877510,2023-03-01,05:10:11,1,285,SJ39
1,13177467,-13.524529,-71.977527,2023-03-01,06:04:51,1,298,SJ34
2,13189674,-13.534183,-71.980709,2023-03-01,07:22:21,1,7,SJ29
3,13192729,-13.551146,-71.986928,2023-03-01,07:39:04,1,202,SJ21
4,13193272,-13.548642,-71.881010,2023-03-01,07:41:52,1,107,SJ34
...,...,...,...,...,...,...,...,...
1685065,17241845,-13.525440,-71.975079,2023-03-25,08:50:24,2,288,SJ24
1685066,17253753,-13.551121,-71.989621,2023-03-25,10:01:56,2,253,SJ16
1685067,17257644,-13.543207,-71.897785,2023-03-25,10:26:10,2,126,SJ24
1685068,17266468,-13.532774,-71.921865,2023-03-25,11:14:26,1,258,SJ43


In [6]:
# preparar el dataset para iterar en todos los posibles samples
samples_df = monitoringdf.copy()

# Crear columna sample para identificar los samples, utilizando la fecha vehiculo y lapso, todo ello encapsulado en un string
samples_df['sample'] = samples_df.apply(lambda row: f"{row['date']} {row['vehicle_id']} {row['lap']}" , axis=1)
samples_dict = {k:v for v, k in enumerate(samples_df['sample'].unique())}
samples_df['sample'] = samples_df['sample'].apply(lambda sample: samples_dict[sample])
samples_df.sort_values(by='sample', inplace=True)
print('nro posibles samples:', len(samples_dict))

# convertir date and time en el formato datetime
samples_df['datetime'] = samples_df['date'] + ' ' + samples_df['time']
samples_df['datetime'] = pd.to_datetime(samples_df['datetime'], format='%Y-%m-%d %H:%M:%S')
samples_df.drop(columns=['date', 'time'], inplace=True)

samples_df

nro posibles samples: 5015


,id,latitude,longitude,lap,navigation,vehicle_id,sample,datetime
28221,13178603,-13.544752,-71.982983,1,57,SJ39,0,2023-03-01 06:13:58
13774,13186169,-13.545547,-71.887602,1,196,SJ39,0,2023-03-01 07:03:10
76431,13184526,-13.535995,-71.908207,1,124,SJ39,0,2023-03-01 06:54:00
64953,13175946,-13.530703,-71.981708,1,78,SJ39,0,2023-03-01 05:48:32
39724,13174976,-13.520454,-71.969145,1,279,SJ39,0,2023-03-01 05:35:16
...,...,...,...,...,...,...,...,...
1680336,17020031,-13.553743,-71.859497,0,104,SJ32,5014,2023-03-24 05:14:13
1677151,17019900,-13.551959,-71.868857,0,106,SJ32,5014,2023-03-24 05:11:45
1680073,17020021,-13.553554,-71.860493,0,97,SJ32,5014,2023-03-24 05:14:01
1678733,17019966,-13.552933,-71.864613,0,97,SJ32,5014,2023-03-24 05:12:56


# Funciones para calcular el travel time

In [ ]:
from haversine import haversine as hvs, Unit
import numpy as np

# FUNCION: 
def searchStopsPoints(row, stops, r_nav=45, r_dtec=60):
    # verificar la navegacion y el radio de distancia
    stops = stops[abs(stops['navigation'] - row['navigation']) <= r_nav]
    stops['dis'] = stops['latitude_longitude'].apply(lambda stop: round(hvs(row['latitude_longitude'], stop, unit=Unit.METERS), 2))
    stops = stops[stops['dis'] <= r_dtec]
    
    # preparar el output
    row['stop'] = str(stops.loc[stops['dis'].idxmin(), 'id']) if len(stops) != 0 else np.nan
    row['dis'] = float(stops.loc[stops['dis'].idxmin(), 'dis']) if len(stops) != 0 else 0    
    #row['long_lati'] = row['latitude_longitude']
    return row

# FUNCION: CALCULA EL TIEMPO EN SEG DE UN PUNTO CON SU ANTERIOR
def time_travel(row, df):
    if {row['init_stop'], row['end_stop']}.issubset(set(df.index)):
        # print('if')
        ti, te =  df.loc[row['init_stop'], 'datetime'], df.loc[row['end_stop'], 'datetime']
        row['date_time_init'] = ti
        row['date_time_end'] = te
        time = abs((ti - te).total_seconds())
        
        #row['time_travel'] =  time if time < 300 else np.nan
        row['time_travel'] =  time
        row['init_idp'], row['end_idp'] = df.loc[row['init_stop'], 'id'], df.loc[row['end_stop'], 'id']

        row['ini_lat'] = df.loc[row['init_stop'], 'latitude_longitude']
        row['end_lat'] = df.loc[row['end_stop'], 'latitude_longitude']

        row['veh'] = df.loc[row['init_stop'], 'vehicle_id']
        row['lap'] = df.loc[row['init_stop'], 'lap']
        row['date'] = df.loc[row['init_stop'], 'date']

    else:
        # print('else')
        row['date_time_init'] = np.nan
        row['date_time_end'] = np.nan
        row['time_travel'] = np.nan
        row['init_idp'] = np.nan
        row['end_idp'] = np.nan
        row['veh'] = np.nan
        row['lap'] = np.nan
        row['date'] = np.nan
        
    return row

# Crear secuencias y calcular tiempos de viaje

In [7]:
metrics_df = pd.DataFrame(columns=['id', 'sample'])

c = 0
for id, sample_df in samples_df.groupby('sample'):
    name = f'{str(sample_df.iloc[0, 6]).split()[0]} {sample_df.iloc[0, 5]} {sample_df.iloc[0, 3]}'
    if(sample_df.shape[0] < 5):
        print(name, f'- {sample_df.shape}')
    c += 1
c

166 SJ41 0 - (4, 8)
262 SJ25 0 - (1, 8)
265 SJ40 5 - (3, 8)
283 SJ08 5 - (1, 8)
298 SJ42 0 - (2, 8)
330 SJ17 0 - (1, 8)
335 SJ09 0 - (1, 8)
337 SJ07 5 - (2, 8)
532 SJ43 0 - (3, 8)
632 SJ43 5 - (3, 8)
633 SJ35 0 - (1, 8)
707 SJ26 5 - (2, 8)
741 SJ29 0 - (1, 8)
743 SJ40 4 - (1, 8)
762 SJ42 0 - (1, 8)
770 SJ33 0 - (2, 8)
774 SJ43 0 - (1, 8)
800 SJ06 5 - (1, 8)
802 SJ08 3 - (3, 8)
807 SJ19 0 - (4, 8)
820 SJ25 6 - (1, 8)
824 SJ34 5 - (1, 8)
868 SJ35 0 - (4, 8)
1098 SJ36 1 - (3, 8)
1115 SJ13 0 - (3, 8)
1191 SJ24 0 - (3, 8)
1211 SJ42 6 - (3, 8)
1214 SJ21 0 - (2, 8)
1232 SJ09 0 - (4, 8)
1239 SJ19 5 - (2, 8)
1241 SJ19 0 - (2, 8)
1242 SJ41 0 - (4, 8)
1258 SJ08 7 - (1, 8)
1286 SJ01 0 - (2, 8)
1293 SJ34 6 - (1, 8)
1295 SJ21 0 - (1, 8)
1301 SJ15 5 - (1, 8)
1474 SJ10 6 - (4, 8)
1548 SJ33 0 - (3, 8)
1644 SJ25 6 - (1, 8)
1657 SJ18 5 - (1, 8)
1659 SJ05 5 - (1, 8)
1662 SJ29 0 - (2, 8)
1689 SJ11 0 - (1, 8)
1691 SJ39 0 - (2, 8)
1704 SJ18 1 - (1, 8)
1714 SJ26 0 - (3, 8)
1730 SJ29 0 - (1, 8)
2005 SJ43 1 - (

5015

In [8]:
samples_df.shape

(1685070, 8)

In [9]:
samples_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1685070 entries, 28221 to 1679049
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   id          1685070 non-null  int64         
 1   latitude    1685070 non-null  float64       
 2   longitude   1685070 non-null  float64       
 3   lap         1685070 non-null  int64         
 4   navigation  1685070 non-null  int64         
 5   vehicle_id  1685070 non-null  object        
 6   sample      1685070 non-null  int64         
 7   datetime    1685070 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 115.7+ MB


In [ ]:
sample_df_tmp = samples_df.copy()
sample_df_tmp['date'] = sample_df_tmp['datetime'].apply(lambda dt: str(dt).split(' ')[0])
sample_df_tmp['latitude_longitude'] = sample_df_tmp.apply(lambda row: (row['latitude'], row['longitude']), axis=1)

In [14]:
sample_df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1685070 entries, 28221 to 1679049
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   id                  1685070 non-null  int64         
 1   latitude            1685070 non-null  float64       
 2   longitude           1685070 non-null  float64       
 3   lap                 1685070 non-null  int64         
 4   navigation          1685070 non-null  int64         
 5   vehicle_id          1685070 non-null  object        
 6   sample              1685070 non-null  int64         
 7   datetime            1685070 non-null  datetime64[ns]
 8   date                1685070 non-null  object        
 9   latitude_longitude  1685070 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(4), object(3)
memory usage: 141.4+ MB


In [ ]:
# iterar en todos los posibles
n_descartados = 0
list_err = []
list_samplesdf = []
date = ''
bt = 0

# Se ordena los posibles samples de manera ascendente
for ids, pointsdf in sample_df_tmp.groupby('sample'): 
    # Seguir procesamiento
    if date != pointsdf['date'].unique()[0]:
        if bt == 1:
            break
        bt += 1

        date = pointsdf['date'].unique()[0]
        print(date)
        
    # ordenar la secuencia por datetime
    pointsdf.sort_values(by='datetime', inplace=True)

    stopsPointsdf = pointsdf.apply(lambda row: searchStopsPoints(row, allbusstop_df), axis=1)

In [12]:
# # iterar en todos los posibles
# n_descartados = 0
# list_err = []
# list_samplesdf = []
# date = ''
# bt = 0

# # Se ordena los posibles samples de manera ascendente
# for ids, pointsdf in sample_df_tmp.groupby('sample'): 

#     # Seguir procesamiento
#     if date != pointsdf['date'].unique()[0]:
#         if (bt == 1):
#             break
#         bt+=1

#         date = pointsdf['date'].unique()[0]
#         print(date)

#     # ordenar la secuencia por datetime
#     pointsdf.sort_values(by='datetime', inplace=True)
    
#     stopsPointsdf = pointsdf.apply(lambda row: searchStopsPoints(row, allbusstop_df), axis=1)

#     stopsPointsdf = stopsPointsdf[stopsPointsdf['stop'].notna()].reset_index()

#     # Seleccionar un unico punto por stops
#     stopsPointsdf.sort_values(by=['stop', 'dis'], ascending=True, inplace=True, ignore_index=True, key=np.int64)
#     stopsPointsdf.drop_duplicates(subset=['stop'], keep='first', inplace=True, ignore_index=True)

#     # filtrar los stop sin puntos 
#     missing_stops = list(set(allbusstop_df['id']) - set(stopsPointsdf['stop'].unique()))

#     # verificar la cantidad de puntos filtrados
#     if len(missing_stops) < 24:
#         # verificar la secuencialidad de los datetime
#         byS = list(stopsPointsdf.sort_values(by='stop')['stop'])
#         byT = list(stopsPointsdf.sort_values(by='datetime')['stop'])
#         if byS == byT:
#             # crear sample y calcular el travel_time
#             sampledf = links_df.copy()
#             stopsPointsdf.set_index('stop', inplace=True)
#             sampledf = sampledf.apply(lambda row: time_travel(row, stopsPointsdf), axis=1)
            
#             # crear el id_sample
#             list_samplesdf.append(sampledf)
#         else:
#             # registrar sample defectoso
#             list_err.append(stopsPointsdf)
#             print(' ', ids)
#     else:
#         n_descartados += 1

# # mostrar nro de samples encontrados
# print('='*100)
# print(f"{len(list_samplesdf)} Samples buenos")
# print(f"{len(list_err)} Samples con errores de continuidad de tiempo")
# print(f"{n_descartados} Samples descartados por falta de puntos")

In [ ]:
list_samplesdf[0].columns

Index(['id_sample', 'date', 'date_time_end', 'date_time_init', 'end_idp',
       'end_lat', 'end_name', 'end_stop', 'id_link', 'ini_lat', 'init_idp',
       'init_name', 'init_stop', 'lap', 'link', 'time_travel', 'veh'],
      dtype='object')

In [ ]:
columnas_ord = ['id_sample', 'date', 'date_time_end', 'date_time_init', 'end_idp', 'end_name', 'end_stop', 'id_link', 'init_idp',
       'init_name', 'init_stop', 'lap', 'link', 'time_travel', 'veh', 'ini_lat',
       'end_lat']

In [ ]:
list_samplesdf[109][columnas_ord]

,id_sample,date,date_time_end,date_time_init,end_idp,end_name,end_stop,id_link,init_idp,init_name,init_stop,lap,link,time_travel,veh,ini_lat,end_lat
0,109,2023-03-01,2023-03-01 14:37:49,2023-03-01 14:04:39,13277666.0,primer stop,101,0,13271047.0,inicio,100,3.0,100:101,1990.0,SJ43,"(-13.5550828, -71.8559498)","(-13.5550275, -71.8570669)"
1,109,2023-03-01,2023-03-01 14:39:56,2023-03-01 14:37:49,13278087.0,segundo stop,102,1,13277666.0,primer stop,101,3.0,101:102,127.0,SJ43,"(-13.5550275, -71.8570669)","(-13.5532644, -71.8622568)"
2,109,2023-03-01,2023-03-01 14:41:46,2023-03-01 14:39:56,13278467.0,Ccollana,103,2,13278087.0,segundo stop,102,3.0,102:103,110.0,SJ43,"(-13.5532644, -71.8622568)","(-13.552325, -71.8672449)"
3,109,2023-03-01,2023-03-01 14:42:30,2023-03-01 14:41:46,13278612.0,Puente Huaccoto,104,3,13278467.0,Ccollana,103,3.0,103:104,44.0,SJ43,"(-13.552325, -71.8672449)","(-13.5516429, -71.8699255)"
4,109,2023-03-01,2023-03-01 14:43:59,2023-03-01 14:42:30,13278918.0,Kayra,105,4,13278612.0,Puente Huaccoto,104,3.0,104:105,89.0,SJ43,"(-13.5516429, -71.8699255)","(-13.5505811, -71.8741039)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,109,2023-03-01,2023-03-01 17:17:27,2023-03-01 17:17:15,13309590.0,Tapia,250,149,13309557.0,Posta,249,3.0,249:250,12.0,SJ43,"(-13.5490034, -71.8797405)","(-13.5492024, -71.8788978)"
150,109,2023-03-01,2023-03-01 17:18:16,2023-03-01 17:17:27,13309738.0,Forestal,251,150,13309590.0,Tapia,250,3.0,250:251,49.0,SJ43,"(-13.5492024, -71.8788978)","(-13.5499915, -71.8762344)"
151,109,2023-03-01,2023-03-01 17:18:40,2023-03-01 17:18:16,13309820.0,Kayra,252,151,13309738.0,Forestal,251,3.0,251:252,24.0,SJ43,"(-13.5499915, -71.8762344)","(-13.5505434, -71.8744531)"
152,109,2023-03-01,2023-03-01 17:20:18,2023-03-01 17:18:40,13310127.0,Puente,253,152,13309820.0,Kayra,252,3.0,252:253,98.0,SJ43,"(-13.5505434, -71.8744531)","(-13.5515743, -71.8703316)"


In [ ]:
list_samplesdf[109][columnas_ord].to_csv('sample_ida_vuela.csv', index=False)